# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [ ]:
import torch
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, w_true, n_features, n_objects):
        self.X = (torch.rand(n_objects, n_features) - 0.5) * 10
        self.X *= (torch.arange(n_features) * 2 + 1)
        self.Y = self.X @ w_true + torch.randn(n_objects)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, item):
        return self.X[item], self.Y[item]

In [ ]:
n_features = 2
n_objects = 300
w_true = torch.randn(n_features)
dataset = CustomDataset(w_true, n_features, n_objects)

In [ ]:
len(dataset)

300

In [ ]:
dataset[0]

(tensor([1.0584, 7.6576]), tensor(-3.6397))

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`.

Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` (серию) и `i`-ую строку из `obj.y` (строку).
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

- [ ] Проверено на семинаре

In [ ]:
class BankDatasetBase(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        self.X = data.drop(columns=['y'])
        self.y = data['y']

    def __getitem__(self, idx: int) -> tuple:
        return self.X.iloc[idx], self.y.iloc[idx]

    def __len__(self) -> int:
        return len(self.X)

In [ ]:
import pandas as pd
df = pd.read_csv('bank-full.csv', sep=',')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
bank_dataset = BankDatasetBase(data=df)
len(bank_dataset)

45211

In [ ]:
bank_dataset[0]

(age                  58
 job          management
 marital         married
 education      tertiary
 default              no
 balance            2143
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            261
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 0, dtype: object,
 'no')

<p class="task" id="2"></p>

2\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательные аргументы `transform` и `target_transform`;
* если аргумент `transform` был передан, то при получении `i`-го элемента, нужно вызвать `transform(x)` и вернуть полученный результат.
* если аргумент `target_transform` был передан, то при получении `i`-го элемента, нужно вызвать `target_transform(y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность (без передачи `target_transform` и `transform`).

- [ ] Проверено на семинаре

In [ ]:
from typing import Callable

class BankDataset(Dataset):
    def __init__(
            self,
            data: pd.DataFrame,
            transform: Callable | None = None,
            target_transform: Callable | None = None
    ) -> None:
        self.X = data.drop(columns=['y'])
        self.y = data['y']
        self.transform = transform
        self.target_transform = target_transform
    def __getitem__(self, idx: int) -> tuple:
        x = self.X.iloc[idx]
        y = self.y.iloc[idx]

        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)

        return x, y

    def __len__(self) -> int:
        return len(self.X)

In [ ]:
df = pd.read_csv('bank-full.csv', sep=',')
bank_dataset_no_transform = BankDataset(data=df)
len(bank_dataset_no_transform)

45211

In [ ]:
bank_dataset_no_transform[0]

(age                  58
 job          management
 marital         married
 education      tertiary
 default              no
 balance            2143
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            261
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 0, dtype: object,
 'no')

<p class="task" id="3"></p>

3\. Опишите класс `OrdinalEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (признаки) и возвращает набор признаков, в котором нечисловые характеристики закодированы целыми числами;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `OrdinalEncoderTransform`.

- [ ] Проверено на семинаре

In [ ]:
class Transform:
    pass

In [ ]:
class OrdinalEncoderTransform:
    def __init__(self, category_columns: list[str]) -> None:
        self.category_columns = category_columns
        self.encoding_dict = {}

    def __call__(self, x: pd.Series) -> pd.Series:
        x_transformed = x.copy()

        for col in self.category_columns:
            if col in x.index:
                value = x[col]
                if col not in self.encoding_dict:
                    self.encoding_dict[col] = {}
                if value not in self.encoding_dict[col]:
                    self.encoding_dict[col][value] = len(self.encoding_dict[col])

                x_transformed[col] = self.encoding_dict[col][value]

        return x_transformed

    def get_encoding_info(self) -> dict:
        return self.encoding_dict.copy()

In [ ]:
df = pd.read_csv('bank-full.csv', sep=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [ ]:
category_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
encoder = OrdinalEncoderTransform(category_columns)

bank_dataset_encoded = BankDataset(data=df, transform=encoder)

indices_to_check = [0, 24, 100]
for idx in indices_to_check:
    print(f"строка {idx}")
    print("До кодирования:")
    print(bank_dataset_no_transform[idx][0][category_columns])
    print("После кодирования:")
    print(bank_dataset_encoded[idx][0][category_columns])

строка 0
До кодирования:
job          management
marital         married
education      tertiary
default              no
housing             yes
loan                 no
contact         unknown
month               may
poutcome        unknown
Name: 0, dtype: object
После кодирования:
job          0
marital      0
education    0
default      0
housing      0
loan         0
contact      0
month        0
poutcome     0
Name: 0, dtype: object
строка 24
До кодирования:
job          retired
marital      married
education    primary
default           no
housing          yes
loan             yes
contact      unknown
month            may
poutcome     unknown
Name: 24, dtype: object
После кодирования:
job          1
marital      0
education    1
default      0
housing      0
loan         1
contact      0
month        0
poutcome     0
Name: 24, dtype: object
строка 100
До кодирования:
job          blue-collar
marital          married
education      secondary
default               no
housing        

In [ ]:
print("Итоговое кодирование")
print(encoder.get_encoding_info())

Итоговое кодирование
{'job': {'management': 0, 'retired': 1, 'blue-collar': 2}, 'marital': {'married': 0}, 'education': {'tertiary': 0, 'primary': 1, 'secondary': 2}, 'default': {'no': 0}, 'housing': {'yes': 0}, 'loan': {'no': 0, 'yes': 1}, 'contact': {'unknown': 0}, 'month': {'may': 0}, 'poutcome': {'unknown': 0}}


<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (строку) и возвращает целое число, соответствующее этой строке;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объекта в качестве аргумента `target_transform` объект класса `LabelEncoderTransform`.

- [ ] Проверено на семинаре

In [ ]:
class LabelEncoderTransform:
    def __init__(self) -> None:
        self.encoding_dict = {}
        self.next_index = 0

    def __call__(self, label: str) -> int:
        if label not in self.encoding_dict:
            self.encoding_dict[label] = self.next_index
            self.next_index += 1
        return self.encoding_dict[label]

    def get_encoding_info(self) -> dict:
        return self.encoding_dict.copy()

In [ ]:
label_encoder = LabelEncoderTransform()
bank_dataset_label_encoded = BankDataset(
    data=df,
    target_transform=label_encoder
)

for i in range(len(bank_dataset_label_encoded)):
    x, y_encoded = bank_dataset_label_encoded[i]
    x, y_original = bank_dataset_no_transform[i]
    print(f"Элемент {i}: y до:'{y_original}', после:{y_encoded}")

print("Информация о кодировании меток:")
print(label_encoder.get_encoding_info())


Выходные данные были обрезаны до нескольких последних строк (5000).
Элемент 40213: y до:'no', после:0
Элемент 40214: y до:'no', после:0
Элемент 40215: y до:'no', после:0
Элемент 40216: y до:'no', после:0
Элемент 40217: y до:'yes', после:1
Элемент 40218: y до:'yes', после:1
Элемент 40219: y до:'yes', после:1
Элемент 40220: y до:'yes', после:1
Элемент 40221: y до:'no', после:0
Элемент 40222: y до:'yes', после:1
Элемент 40223: y до:'no', после:0
Элемент 40224: y до:'no', после:0
Элемент 40225: y до:'no', после:0
Элемент 40226: y до:'yes', после:1
Элемент 40227: y до:'no', после:0
Элемент 40228: y до:'yes', после:1
Элемент 40229: y до:'no', после:0
Элемент 40230: y до:'no', после:0
Элемент 40231: y до:'yes', после:1
Элемент 40232: y до:'no', после:0
Элемент 40233: y до:'yes', после:1
Элемент 40234: y до:'no', после:0
Элемент 40235: y до:'no', после:0
Элемент 40236: y до:'no', после:0
Элемент 40237: y до:'no', после:0
Элемент 40238: y до:'no', после:0
Элемент 40239: y до:'no', после:0
Элеме

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает на вход серию или фрейм и возвращает тензор.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании преобразования `Compose` список из объектов LabelEncoderTransform и ToTensor.

- [ ] Проверено на семинаре

In [ ]:
import torch as th
from typing import Any

class ToTensor(Transform):
    def __call__(self, X: pd.Series | int) -> th.Tensor:
        pass

class Compose(Transform):
    def __init__(self, transforms: list[Transform]) -> None:
        pass

    def __call__(self, X: Any) -> Any:
        pass

In [ ]:
import torch as th
from typing import Any

class Transform:
    pass

class ToTensor(Transform):
    def __call__(self, x: pd.Series | int) -> th.Tensor:
        if isinstance(x, pd.Series):
            return th.tensor(x.values, dtype=th.float32)
        elif isinstance(x, int):
            return th.tensor(x, dtype=th.int64)

In [ ]:
class Compose(Transform):
    def __init__(self, transforms: list[Transform]) -> None:
        self.transforms = transforms

    def __call__(self, x: Any) -> Any:
        for transform in self.transforms:
            x = transform(x)
        return x

In [ ]:
label_encoder = LabelEncoderTransform()
to_tensor = ToTensor()
target_transforms = Compose(transforms=[label_encoder, to_tensor])

bank_dataset_composed = BankDataset(
    data=df,
    target_transform=target_transforms
)

original_label = df.loc[0, 'y']
_, encoded_label = bank_dataset_composed[0]
print(f"Оригинальная метка: '{original_label}', Преобразованная метка: {encoded_label}")
print(f"Тип данных: {encoded_label.dtype}")
print(f"Словарь кодирования: {label_encoder.get_encoding_info()}")

Оригинальная метка: 'no', Преобразованная метка: 0
Тип данных: torch.int64
Словарь кодирования: {'no': 0}


<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.

- [ ] Проверено на семинаре

In [ ]:
import pandas as pd
import torch as th
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from typing import Callable, Any

class Transform:
    pass

class BankDataset(Dataset):
    def __init__(self, data: pd.DataFrame, transform: Callable | None = None, target_transform: Callable | None = None) -> None:
        self.X = data.drop(columns=['y'])
        self.y = data['y']
        self.transform = transform
        self.target_transform = target_transform
    def __getitem__(self, idx: int) -> tuple:
        x = self.X.iloc[idx]
        y = self.y.iloc[idx]
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
        return x, y
    def __len__(self) -> int:
        return len(self.X)

class OrdinalEncoderTransform(Transform):
    def __init__(self, category_columns: list[str]) -> None:
        self.category_columns = category_columns
        self.encoding_dict = {}
        self.next_code = {col: 0 for col in category_columns}
    def __call__(self, x: pd.Series) -> pd.Series:
        x_transformed = x.copy()
        for col in self.category_columns:
            if col in x_transformed.index:
                category = x_transformed[col]
                if category not in self.encoding_dict.get(col, {}):
                    if col not in self.encoding_dict:
                        self.encoding_dict[col] = {}
                    self.encoding_dict[col][category] = self.next_code[col]
                    self.next_code[col] += 1
                x_transformed[col] = self.encoding_dict[col][category]
        return x_transformed.astype(float)

class LabelEncoderTransform(Transform):
    def __init__(self) -> None:
        self.encoding_dict = {}
        self.next_index = 0
    def __call__(self, label: str) -> int:
        if label not in self.encoding_dict:
            self.encoding_dict[label] = self.next_index
            self.next_index += 1
        return self.encoding_dict[label]
    def get_encoding_info(self) -> dict:
        return self.encoding_dict.copy()

class ToTensor(Transform):
    def __call__(self, x: pd.Series | int) -> th.Tensor:
        if isinstance(x, pd.Series):
            return th.tensor(x.values, dtype=th.float32)
        elif isinstance(x, int):
            return th.tensor(x, dtype=th.int64)
        else:
            raise TypeError("Unsupported data type for ToTensor")

class Compose(Transform):
    def __init__(self, transforms: list[Transform]) -> None:
        self.transforms = transforms
    def __call__(self, x: Any) -> Any:
        for transform in self.transforms:
            x = transform(x)
        return x

df = pd.read_csv('bank-full.csv', sep=',')
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

encoder_features = OrdinalEncoderTransform(category_columns=categorical_cols)
encoder_target = LabelEncoderTransform()
to_tensor = ToTensor()

feature_transforms = Compose(transforms=[encoder_features, to_tensor])
target_transforms = Compose(transforms=[encoder_target, to_tensor])

train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

train_dataset = BankDataset(
    data=train_df,
    transform=feature_transforms,
    target_transform=target_transforms
)

def custom_collate_fn(batch):
    features = [item[0] for item in batch]
    labels = [item[1] for item in batch]

    features_tensor = th.stack(features)
    labels_tensor = th.stack(labels)

    features_tensor_reshaped = features_tensor.view(-1, 2, 8)

    return features_tensor_reshaped, labels_tensor

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=custom_collate_fn
)
batch_features, batch_labels = next(iter(train_loader))

print(f"Размерность тензора признаков: {batch_features.shape}")
print(f"Размерность тензора меток: {batch_labels.shape}")

Размерность тензора признаков: torch.Size([64, 2, 8])
Размерность тензора меток: torch.Size([64])


In [ ]:
batch_features

tensor([[[ 2.9000e+01,  0.0000e+00,  0.0000e+00,  ...,  2.4700e+02,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  7.0000e+00,  0.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 4.1000e+01,  1.0000e+00,  1.0000e+00,  ...,  6.4600e+02,
           1.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  2.9000e+01,  1.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 3.0000e+01,  2.0000e+00,  0.0000e+00,  ...,  1.5670e+03,
           1.0000e+00,  1.0000e+00],
         [ 0.0000e+00,  1.2000e+01,  2.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00]],

        ...,

        [[ 2.7000e+01,  2.0000e+00,  1.0000e+00,  ...,  1.0200e+02,
           0.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  2.6000e+01,  0.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 5.7000e+01,  2.0000e+00,  0.0000e+00,  ...,  1.3410e+03,
           1.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  8.0000e+0